# Test orbit file (LED) reading and Hermite interpolation on the va;lues

In [2]:
import numpy as np
import pandas as pd

In [1]:
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
%matplotlib inline

In [45]:
import pandas as pd

def read_orb(ldrfile):
    """
    Read an LED file and extract the metadata and data into a dictionary and a DataFrame.

    Parameters:
    ldrfile (str): Path to the LED file.

    Returns:
    tuple: A tuple containing the metadata dictionary and the data DataFrame.

    Metadata Dictionary:
    - 'nd' (str): Number of given points in the list.
    - 'iy' (str): Year.
    - 'id' (str): Julian day.
    - 'isec' (str): Seconds of the day.
    - 'idsec' (str): Delta seconds.

    Data DataFrame:
    - 'iy' (int): Year.
    - 'id' (int): Julian day.
    - 'isec' (float): Seconds of the day.
    - 'px' (float): X-coordinate.
    - 'py' (float): Y-coordinate.
    - 'pz' (float): Z-coordinate.
    - 'vx' (float): X-velocity.
    - 'vy' (float): Y-velocity.
    - 'vz' (float): Z-velocity.
    """

    with open(ldrfile, 'r') as file:
        first_line = file.readline()
    columns = ['nd', 'iy', 'id', 'isec', 'idsec']
    meta = dict(zip(columns, first_line.split()))

    # Define the column headers based on the structure in the code
    headers = ['iy', 'id', 'isec', 'px', 'py', 'pz', 'vx', 'vy', 'vz']
    # Read the file with pandas, specifying space as the delimiter and skipping the first row
    df = pd.read_csv(ldrfile, delimiter=' ', header=None, skiprows=1, names=headers, index_col=False)
    df.attrs = meta
    
    return df

orb = read_orb('S1_20150521_ALL_F1.LED')

print(orb.attrs)
orb

{'nd': '281', 'iy': '2015', 'id': '140', 'isec': '48064.000000', 'idsec': '10.000'}


,iy,id,isec,px,py,pz,vx,vy,vz
0,2015,140,48064.0,1.941649e+06,2.690324e+06,6.239974e+06,-1656.759849,-6599.833496,3353.575432
1,2015,140,48074.0,1.924925e+06,2.624189e+06,6.273157e+06,-1688.050184,-6627.146750,3283.006276
2,2015,140,48084.0,1.907889e+06,2.557784e+06,6.305632e+06,-1719.190204,-6653.671699,3212.068735
3,2015,140,48094.0,1.890542e+06,2.491118e+06,6.337397e+06,-1750.175288,-6679.405648,3140.770806
4,2015,140,48104.0,1.872886e+06,2.424198e+06,6.368447e+06,-1781.000831,-6704.345996,3069.120523
...,...,...,...,...,...,...,...,...,...
276,2015,140,50824.0,-3.004528e+06,-3.385614e+06,-5.445894e+06,2234.108457,5529.166846,-4673.002701
277,2015,140,50834.0,-2.981980e+06,-3.330151e+06,-5.492317e+06,2275.548048,5563.291422,-4611.613522
278,2015,140,50844.0,-2.959018e+06,-3.274350e+06,-5.538124e+06,2316.780830,5596.733029,-4549.711427
279,2015,140,50854.0,-2.935645e+06,-3.218219e+06,-5.583310e+06,2357.801267,5629.488363,-4487.303403


In [31]:
orb.isec[:10].values

array([48064., 48074., 48084., 48094., 48104., 48114., 48124., 48134.,
       48144., 48154.])

In [29]:
orb.px[:10].values

array([1941649.304103, 1924925.130555, 1907888.801379, 1890541.842833,
       1872885.827312, 1854922.373199, 1836653.144729, 1818079.851828,
       1799204.249944, 1780028.139872])

In [30]:
orb.vx[:10].values

array([-1656.759849, -1688.050184, -1719.190204, -1750.175288,
       -1781.000831, -1811.662241, -1842.15494 , -1872.474369,
       -1902.615983, -1932.575255])

In [40]:
import numpy as np
from numpy.polynomial.hermite import hermfit, hermval

# Define the data points
x = orb.isec[:10].values
y = orb.px[:10].values
z = orb.vx[:10].values

# Fit the Hermite polynomial
coeff = hermfit(x, y, 5)

for test_x in [48070.000000, 48078.000000, 48086.000000, 48092.000000, 48102.000000,
        48118.000000, 48128.000000, 48138.000000, 48150.000000, 48160.000000]:
    # Perform the interpolation
    y_interpolated = hermval(test_x, coeff)
    # Output the interpolated value
    print(f'Interpolated value at x={test_x}: ', y_interpolated)

Interpolated value at x=48070.0:  1931652.3436279297
Interpolated value at x=48078.0:  1918147.973815918
Interpolated value at x=48086.0:  1904444.2108154297
Interpolated value at x=48092.0:  1894036.0103759766
Interpolated value at x=48102.0:  1876441.6782836914
Interpolated value at x=48118.0:  1847651.2810668945
Interpolated value at x=48128.0:  1829260.2188110352
Interpolated value at x=48138.0:  1810565.7890014648
Interpolated value at x=48150.0:  1787734.5283203125
Interpolated value at x=48160.0:  1768378.997619629


In [44]:
%timeit hermval(test_x, coeff)

1.35 µs ± 20.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
